In [1]:
#import some stuff
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import os
from tqdm import tqdm
import glob
import time
import xgboost as xgb
from sklearn.model_selection import train_test_split #use this?
from scipy import stats
from sklearn.metrics import log_loss, mean_squared_error, mean_squared_log_error
from sklearn.model_selection import GridSearchCV #use random search?

In [2]:
pd.set_option('display.max_columns', 500)

In [3]:
cities = pd.read_csv('Cities.csv')
conferences = pd.read_csv('Conferences.csv')
MConfTGames = pd.read_csv('MConferenceTourneyGames.csv')
MGameCities = pd.read_csv('MGameCities.csv')
MMasseyO = pd.read_csv('MMasseyOrdinals.csv')
MTCompactResults = pd.read_csv('MNCAATourneyCompactResults.csv')
MTDetailedResults = pd.read_csv('MNCAATourneyDetailedResults.csv')
MTSeedRoundSlots = pd.read_csv('MNCAATourneySeedRoundSlots.csv')
MTSeeds = pd.read_csv('MNCAATourneySeeds.csv')
MTSlots = pd.read_csv('MNCAATourneySlots.csv')
RegCompactResults = pd.read_csv('MRegularSeasonCompactResults.csv')
RegDetailedResults = pd.read_csv('MRegularSeasonDetailedResults.csv')
MSampleStage1 = pd.read_csv('MSampleSubmissionStage1.csv')
Mszns = pd.read_csv('MSeasons.csv')
MSecTourneyComRes = pd.read_csv('MSecondaryTourneyCompactResults.csv')
MSecTourneyTeams = pd.read_csv('MSecondaryTourneyTeams.csv')
MTeamConfs = pd.read_csv('MTeamConferences.csv')
#MTeamSpellings = pd.read_csv('MTeamSpellings.csv') #can't read for some reason
MTeamCoaches = pd.read_csv('MTeamCoaches.csv')
MTeams = pd.read_csv('MTeams.csv')

In [4]:
cities.head(1)

,CityID,City,State
0,4001,Abilene,TX


In [5]:
conferences.head(1)

,ConfAbbrev,Description
0,a_sun,Atlantic Sun Conference


In [6]:
MConfTGames.head(1)

,Season,ConfAbbrev,DayNum,WTeamID,LTeamID
0,2001,a_sun,121,1194,1144


In [7]:
MGameCities.head(1)

,Season,DayNum,WTeamID,LTeamID,CRType,CityID
0,2010,7,1143,1293,Regular,4027


In [8]:
MMasseyO.head(1)

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
0,2003,35,SEL,1102,159


In [62]:
MTCompactResults.head(1)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0


In [10]:
MTDetailedResults.head(1)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,134,1421,92,1411,84,N,1,32,69,11,29,17,26,14,30,17,12,5,3,22,29,67,12,31,14,31,17,28,16,15,5,0,22


In [11]:
MTSeedRoundSlots.head(1)

,Seed,GameRound,GameSlot,EarlyDayNum,LateDayNum
0,W01,1,R1W1,136,137


In [12]:
MTSeeds.head(1)

,Season,Seed,TeamID
0,1985,W01,1207


In [13]:
MTSlots.head(1)

,Season,Slot,StrongSeed,WeakSeed
0,1985,R1W1,W01,W16


In [14]:
RegCompactResults.head(1)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0


In [15]:
RegDetailedResults.head(1)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20


In [16]:
Mszns.head(1)

,Season,DayZero,RegionW,RegionX,RegionY,RegionZ
0,1985,10/29/1984,East,West,Midwest,Southeast


In [17]:
MSecTourneyComRes.head(1)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,SecondaryTourney
0,1985,136,1151,67,1155,65,H,0,NIT


In [18]:
MSecTourneyTeams.head(1)

,Season,SecondaryTourney,TeamID
0,1985,NIT,1108


In [19]:
MTeamConfs.head(1)

,Season,TeamID,ConfAbbrev
0,1985,1102,wac


In [20]:
MTeamCoaches.head(1)

,Season,TeamID,FirstDayNum,LastDayNum,CoachName
0,1985,1102,0,154,reggie_minton


In [21]:
MTeams.head(1)

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2021


In [22]:
RegCompactResults

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0
...,...,...,...,...,...,...,...,...
166875,2020,128,1204,81,1209,62,A,0
166876,2020,128,1402,85,1111,68,H,0
166877,2020,128,1299,86,1224,77,N,0
166878,2020,128,1393,81,1314,53,N,0


In [23]:
teamWs = RegCompactResults[['Season','WTeamID','WScore']].groupby(by=['Season','WTeamID']).count()
teamWs = teamWs.reset_index()
teamWs = teamWs.rename(columns={'WScore':'Wins'}) 
teamWs = teamWs.merge(MTeams[['TeamID','TeamName']],how='left',left_on='WTeamID',right_on='TeamID')
teamLs = RegCompactResults[['Season','LTeamID','LScore']].groupby(by=['Season','LTeamID']).count()
teamLs = teamLs.reset_index()
teamLs = teamLs.rename(columns={'LScore':'Losses'})
teamLs = teamLs.merge(MTeams[['TeamID','TeamName']],how='left',left_on='LTeamID',right_on='TeamID')
teamWs = teamWs.drop(columns='WTeamID')
teamLs = teamLs.drop(columns='LTeamID')

In [65]:
RegCompactResults[['Season','WTeamID','WScore']].groupby(by=['Season','WTeamID']).count().reset_index()

,Season,WTeamID,WScore
0,1985,1102,5
1,1985,1103,9
2,1985,1104,21
3,1985,1106,10
4,1985,1108,19
...,...,...,...
11575,2020,1463,21
11576,2020,1464,16
11577,2020,1465,17
11578,2020,1466,11


In [24]:
teamWs.loc[teamWs['Season'] == 2020].sort_values(by='Wins',ascending=False).head(1)

,Season,Wins,TeamID,TeamName
11331,2020,31,1211,Gonzaga


In [25]:
teamLs.loc[teamLs['Season'] == 2020].sort_values(by='Losses',ascending=False).head(1)

,Season,Losses,TeamID,TeamName
11374,2020,28,1244,Kennesaw


In [26]:
MMasseyO.loc[MMasseyO['Season'] == 2003,'RankingDayNum'].unique() #use POM or HOL or MAS - 2003 earliest season

array([ 35,  37,  42,  43,  44,  49,  51,  56,  57,  58,  62,  63,  64,
        65,  69,  70,  71,  72,  76,  77,  79,  83,  84,  85,  86,  90,
        91,  93,  98,  99, 100, 105, 106, 107, 111, 112, 114, 117, 118,
       119, 120, 121, 126, 127, 128, 133], dtype=int64)

In [27]:
MMasseyO.loc[(MMasseyO['SystemName'] == 'MAS')].head(1)

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
1987,2003,37,MAS,1102,90


In [28]:
smallRDR = RegDetailedResults[['Season','WTeamID','WScore','WFGM','WFGA','WFGM3','WFGA3','WFTM','WFTA',
                              'LTeamID','LScore','LFGM','LFGA','LFGM3','LFGA3','LFTM','LFTA']].copy()
#this is for just shooting stuff

In [29]:
smallRDR['Games'] = np.zeros(len(smallRDR))
gamesPlayed = smallRDR[['Season','WTeamID','Games']].groupby(by=['Season','WTeamID']).count()
gamesPlayed2 = smallRDR[['Season','LTeamID','Games']].groupby(by=['Season','LTeamID']).count()
gamesPlayed = gamesPlayed.reset_index()
gamesPlayed2 = gamesPlayed2.reset_index()
gamesPlayed = gamesPlayed.merge(gamesPlayed2,how='outer',left_on=['Season','WTeamID'],right_on=['Season','LTeamID'])
gamesPlayed['Games_x'] = gamesPlayed.apply(lambda row: 0 if pd.isnull(row['Games_x']) else row['Games_x'],axis=1)
gamesPlayed['Games_y'] = gamesPlayed.apply(lambda row: 0 if pd.isnull(row['Games_y']) else row['Games_y'],axis=1)
gamesPlayed['TotalGames'] = gamesPlayed['Games_x'] + gamesPlayed['Games_y']
gamesPlayed['WTeamID'] = gamesPlayed.apply(lambda row: row['LTeamID'] if pd.isnull(row['WTeamID']) else row['WTeamID'],axis=1)
gamesPlayed = gamesPlayed[['Season','WTeamID','TotalGames']]
smallRDR = smallRDR.drop(columns=['Games'])
TotalWs = smallRDR.groupby(by=['Season','WTeamID']).sum().reset_index()
TotalLs = smallRDR.groupby(by=['Season','LTeamID']).sum().reset_index()
allgames = TotalWs.merge(TotalLs,how='outer',left_on=['Season','WTeamID'],right_on=['Season','LTeamID'])
allgames = allgames.rename(columns={'WTeamID_x':'WTeamID'})
allgames['WTeamID'] = allgames.apply(lambda row: row['LTeamID_y'] if pd.isnull(row['WTeamID']) else row['WTeamID'],axis=1)
allgames = allgames.merge(gamesPlayed,how='left',on=['Season','WTeamID'])

In [30]:
for col in allgames:
    allgames[col] = allgames.apply(lambda row: 0 if pd.isnull(row[col]) else row[col],axis=1)
for col in allgames:
    if (col[-1] == "x"):
        if col[0] == "W":
            allgames[col[:-2]] = round((allgames[col] + allgames['L' + col[1:-1] + 'y']) / allgames['TotalGames'],3)
        else:
            allgames[col[:-2]] = round((allgames[col] + allgames['W' + col[1:-1] + 'y']) / allgames['TotalGames'],3)

In [31]:
allgames

,Season,WTeamID,WScore_x,WFGM_x,WFGA_x,WFGM3_x,WFGA3_x,WFTM_x,WFTA_x,LTeamID_x,LScore_x,LFGM_x,LFGA_x,LFGM3_x,LFGA3_x,LFTM_x,LFTA_x,LTeamID_y,WTeamID_y,WScore_y,WFGM_y,WFGA_y,WFGM3_y,WFGA3_y,WFTM_y,WFTA_y,LScore_y,LFGM_y,LFGA_y,LFGM3_y,LFGA3_y,LFTM_y,LFTA_y,TotalGames,WScore,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,LTeamID,LScore,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA
0,2003.0,1102.0,825.0,271.0,480.0,120.0,259.0,163.0,249.0,15438.0,638.0,228.0,560.0,57.0,170.0,125.0,174.0,1102.0,21492.0,958.0,312.0,628.0,76.0,178.0,258.0,365.0,778.0,265.0,634.0,99.0,324.0,149.0,230.0,28.0,57.250,19.143,39.786,7.821,20.821,11.143,17.107,1318.929,57.000,19.286,42.429,4.750,12.429,13.679,19.250
1,2003.0,1103.0,1141.0,390.0,720.0,71.0,187.0,290.0,402.0,16243.0,1019.0,358.0,780.0,83.0,265.0,220.0,302.0,1103.0,17734.0,1091.0,392.0,759.0,97.0,231.0,210.0,296.0,986.0,343.0,788.0,76.0,247.0,224.0,296.0,27.0,78.778,27.148,55.852,5.444,16.074,19.037,25.852,1258.407,78.148,27.778,57.000,6.667,18.370,15.926,22.148
2,2003.0,1104.0,1270.0,439.0,992.0,120.0,354.0,272.0,383.0,21910.0,1046.0,376.0,978.0,104.0,335.0,190.0,268.0,1104.0,14004.0,774.0,275.0,576.0,74.0,201.0,150.0,212.0,670.0,234.0,609.0,58.0,202.0,144.0,203.0,28.0,69.286,24.036,57.179,6.357,19.857,14.857,20.929,1282.643,65.000,23.250,55.500,6.357,19.143,12.143,17.143
3,2003.0,1105.0,556.0,179.0,433.0,64.0,157.0,134.0,180.0,8797.0,465.0,161.0,403.0,31.0,122.0,112.0,165.0,1105.0,24227.0,1528.0,541.0,1130.0,132.0,334.0,314.0,472.0,1310.0,455.0,1169.0,133.0,383.0,267.0,388.0,26.0,71.769,24.385,61.615,7.577,20.769,15.423,21.846,1270.154,76.654,27.000,58.962,6.269,17.538,16.385,24.500
4,2003.0,1106.0,888.0,322.0,700.0,76.0,207.0,168.0,270.0,15628.0,753.0,244.0,702.0,52.0,182.0,213.0,302.0,1106.0,19239.0,1032.0,364.0,793.0,82.0,244.0,222.0,313.0,893.0,334.0,848.0,95.0,287.0,130.0,191.0,28.0,63.607,23.429,55.286,6.107,17.643,10.643,16.464,1245.250,63.750,21.714,53.393,4.786,15.214,15.536,21.964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6182,2005.0,1366.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1366.0,34842.0,2274.0,810.0,1751.0,193.0,563.0,461.0,686.0,1623.0,577.0,1650.0,130.0,476.0,339.0,586.0,28.0,57.964,20.607,58.929,4.643,17.000,12.107,20.929,1244.357,81.214,28.929,62.536,6.893,20.107,16.464,24.500
6183,2008.0,1312.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1312.0,37440.0,2233.0,763.0,1611.0,203.0,506.0,504.0,738.0,1622.0,550.0,1546.0,142.0,500.0,380.0,620.0,29.0,55.931,18.966,53.310,4.897,17.241,13.103,21.379,1291.034,77.000,26.310,55.552,7.000,17.448,17.379,25.448
6184,2013.0,1212.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1212.0,34676.0,2138.0,772.0,1683.0,213.0,563.0,381.0,588.0,1391.0,519.0,1436.0,143.0,458.0,210.0,358.0,28.0,49.679,18.536,51.286,5.107,16.357,7.500,12.786,1238.429,76.357,27.571,60.107,7.607,20.107,13.607,21.000
6185,2015.0,1212.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1212.0,33569.0,1992.0,721.0,1546.0,172.0,537.0,378.0,599.0,1380.0,472.0,1322.0,97.0,342.0,339.0,523.0,27.0,51.111,17.481,48.963,3.593,12.667,12.556,19.370,1243.296,73.778,26.704,57.259,6.370,19.889,14.000,22.185


In [32]:
allgames = allgames[['Season','WTeamID','TotalGames','WScore','WFGM','WFGA','WFGM3','WFGA3','WFTM','WFTA',#add 'LScore'
                    'LFGM','LFGA','LFGM3','LFGA3','LFTM','LFTA']]
for col in allgames:
    if col[0] == 'W':
        allgames = allgames.rename(columns={col:'Team'+col[1:]})
    elif col[0] == 'L':
        allgames = allgames.rename(columns={col:'Opp'+col[1:]})

In [34]:
allgames = allgames.rename(columns={'TeamTeamID':'TeamID'})
allgames = allgames.merge(MTeams[['TeamID','TeamName']],how='left',on='TeamID')

In [35]:
allgames.head(1)

,Season,TeamID,TotalGames,TeamScore,TeamFGM,TeamFGA,TeamFGM3,TeamFGA3,TeamFTM,TeamFTA,OppFGM,OppFGA,OppFGM3,OppFGA3,OppFTM,OppFTA,TeamName
0,2003.0,1102.0,28.0,57.25,19.143,39.786,7.821,20.821,11.143,17.107,19.286,42.429,4.75,12.429,13.679,19.25,Air Force


In [36]:
RegDetailedResults.head(1)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20


In [42]:
def getAllAvgs(df):
    df = df.copy()
    df = df.drop(columns=['DayNum','NumOT'])
    df['Games'] = np.zeros(len(df))
    gamesPlayed = df[['Season','WTeamID','Games']].groupby(by=['Season','WTeamID']).count()
    gamesPlayed2 = df[['Season','LTeamID','Games']].groupby(by=['Season','LTeamID']).count()
    gamesPlayed = gamesPlayed.reset_index()
    gamesPlayed2 = gamesPlayed2.reset_index()
    gamesPlayed = gamesPlayed.merge(gamesPlayed2,how='outer',left_on=['Season','WTeamID'],right_on=['Season','LTeamID'])
    gamesPlayed['Games_x'] = gamesPlayed.apply(lambda row: 0 if pd.isnull(row['Games_x']) else row['Games_x'],axis=1)
    gamesPlayed['Games_y'] = gamesPlayed.apply(lambda row: 0 if pd.isnull(row['Games_y']) else row['Games_y'],axis=1)
    gamesPlayed['TotalGames'] = gamesPlayed['Games_x'] + gamesPlayed['Games_y']
    gamesPlayed['WTeamID'] = gamesPlayed.apply(lambda row: row['LTeamID'] if pd.isnull(row['WTeamID']) else row['WTeamID'],axis=1)
    gamesPlayed = gamesPlayed[['Season','WTeamID','TotalGames']]
    df = df.drop(columns=['Games'])
    TotalWs = df.groupby(by=['Season','WTeamID']).sum().reset_index()
    TotalLs = df.groupby(by=['Season','LTeamID']).sum().reset_index()
    allgames = TotalWs.merge(TotalLs,how='outer',left_on=['Season','WTeamID'],right_on=['Season','LTeamID'])
    allgames = allgames.rename(columns={'WTeamID_x':'WTeamID'})
    allgames['WTeamID'] = allgames.apply(lambda row: row['LTeamID_y'] if pd.isnull(row['WTeamID']) else row['WTeamID'],axis=1)
    allgames = allgames.merge(gamesPlayed,how='left',on=['Season','WTeamID'])
    for col in allgames:
        allgames[col] = allgames.apply(lambda row: 0 if pd.isnull(row[col]) else row[col],axis=1)
    for col in allgames:
        if (col[-1] == "x"):
            if col[0] == "W":
                allgames[col[:-2]] = round((allgames[col] + allgames['L' + col[1:-1] + 'y']) / allgames['TotalGames'],3)
            else:
                allgames[col[:-2]] = round((allgames[col] + allgames['W' + col[1:-1] + 'y']) / allgames['TotalGames'],3)
            
    allgames = allgames[['Season','WTeamID','TotalGames','WScore','WFGM','WFGA','WFGM3','WFGA3','WFTM','WFTA',
                         'WOR','WDR','WAst','WTO','WStl','WBlk','WPF','LScore','LFGM','LFGA','LFGM3','LFGA3','LFTM','LFTA',
                        'LOR','LDR','LAst','LTO','LStl','LBlk','LPF']]
            
    for col in allgames:
        if col[0] == 'W':
            allgames = allgames.rename(columns={col:'Team'+col[1:]})
        elif col[0] == 'L':
            allgames = allgames.rename(columns={col:'Opp'+col[1:]})
    
    allgames = allgames.rename(columns={'TeamTeamID':'TeamID'})
    allgames = allgames.merge(MTeams[['TeamID','TeamName']],how='left',on='TeamID')
    return allgames

In [43]:
allAvg = getAllAvgs(RegDetailedResults)

In [44]:
allAvg.head(1)

,Season,TeamID,TotalGames,TeamScore,TeamFGM,TeamFGA,TeamFGM3,TeamFGA3,TeamFTM,TeamFTA,TeamOR,TeamDR,TeamAst,TeamTO,TeamStl,TeamBlk,TeamPF,OppFGM,OppFGA,OppFGM3,OppFGA3,OppFTM,OppFTA,OppOR,OppDR,OppAst,OppTO,OppStl,OppBlk,OppPF,TeamName
0,2003.0,1102.0,28.0,57.25,19.143,39.786,7.821,20.821,11.143,17.107,4.179,16.821,13.0,11.429,5.964,1.786,18.75,19.286,42.429,4.75,12.429,13.679,19.25,9.607,20.143,9.143,12.964,5.429,1.571,18.357,Air Force


In [45]:
just2018Avg = allgames.loc[allgames['Season'] == 2018].copy()
just2018Compact = RegCompactResults.loc[RegCompactResults['Season'] == 2018].copy()
just2018 = RegDetailedResults.loc[RegDetailedResults['Season'] == 2018].copy()

In [46]:
just2018Avg.head(1)

,Season,TeamID,TotalGames,TeamScore,TeamFGM,TeamFGA,TeamFGM3,TeamFGA3,TeamFTM,TeamFTA,OppFGM,OppFGA,OppFGM3,OppFGA3,OppFTM,OppFTA,TeamName
5125,2018.0,1101.0,27.0,70.667,25.519,56.481,6.481,19.963,13.148,18.667,24.778,54.296,6.63,19.741,16.63,23.444,Abilene Chr


In [47]:
just2018['WHome'] = just2018.apply(lambda row: (row['WLoc'] == 'H')*1,axis=1)
just2018['WNeutral'] = just2018.apply(lambda row: (row['WLoc'] == 'N')*1,axis=1)
just2018['WAway'] = just2018.apply(lambda row: (row['WLoc'] == 'A')*1,axis=1)
just2018 = just2018.drop(columns=['WLoc'])

In [48]:
just2018 = just2018[['Season','DayNum','WTeamID','WScore','WFGM','WFGA','WFGM3','WFGA3','WFTM','WFTA',#'NumOT','WHome','WNeutral','WAway',
                              'LTeamID','LScore','LFGM','LFGA','LFGM3','LFGA3','LFTM','LFTA']].copy()

In [49]:
just2018.head(1) #5405 rows

,Season,DayNum,WTeamID,WScore,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,LTeamID,LScore,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA
76636,2018,11,1104,82,26,57,11,27,19,28,1272,70,23,60,2,17,22,36


In [52]:
totalWList = []
totalLList = []
for i in range(len(just2018)):
    WteamsPlayed = []
    LteamsPlayed = []
    row = just2018.iloc[i]
    tempDF = just2018.loc[(just2018['DayNum'] < row['DayNum'])]
    WtempDFws = tempDF.loc[(tempDF['WTeamID'] == row['WTeamID'])].copy()
    WtempDFls = tempDF.loc[(tempDF['LTeamID'] == row['WTeamID'])].copy()
    LtempDFws = tempDF.loc[(tempDF['WTeamID'] == row['LTeamID'])].copy()
    LtempDFls = tempDF.loc[(tempDF['LTeamID'] == row['LTeamID'])].copy()
    WteamsPlayed = WtempDFws['LTeamID'].tolist() + WtempDFls['WTeamID'].tolist()
    LteamsPlayed = LtempDFws['LTeamID'].tolist() + LtempDFls['WTeamID'].tolist()
    totalWList.append(WteamsPlayed)
    totalLList.append(LteamsPlayed)
just2018['WTeamsPlayed'] = totalWList
just2018['LTeamsPlayed'] = totalLList

In [53]:
just2018

,Season,DayNum,WTeamID,WScore,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,LTeamID,LScore,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,WTeamsPlayed,LTeamsPlayed
76636,2018,11,1104,82,26,57,11,27,19,28,1272,70,23,60,2,17,22,36,[],[]
76637,2018,11,1107,69,24,62,4,15,17,19,1233,67,26,59,4,22,11,14,[],[]
76638,2018,11,1112,101,34,57,5,13,28,32,1319,67,23,58,4,13,17,30,[],[]
76639,2018,11,1113,94,34,65,9,30,17,26,1226,74,26,59,9,24,13,22,[],[]
76640,2018,11,1116,95,36,63,9,22,14,22,1359,56,20,61,8,29,8,15,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82036,2018,132,1153,56,20,46,4,14,12,19,1222,55,19,55,7,23,10,12,"[1366, 1441, 1164, 1138, 1350, 1461, 1106, 128...","[1270, 1251, 1448, 1230, 1309, 1116, 1193, 138..."
82037,2018,132,1172,58,19,50,6,18,14,16,1348,57,20,52,8,23,9,17,"[1149, 1423, 1150, 1440, 1103, 1387, 1206, 120...","[1421, 1221, 1371, 1135, 1344, 1158, 1233, 119..."
82038,2018,132,1209,74,25,56,8,17,16,20,1426,61,17,57,4,25,23,28,"[1349, 1186, 1408, 1105, 1285, 1269, 1151, 111...","[1258, 1140, 1441, 1310, 1349, 1317, 1410, 114..."
82039,2018,132,1246,77,25,50,7,16,20,24,1397,72,23,62,10,27,16,20,"[1430, 1436, 1190, 1407, 1236, 1227, 1217, 128...","[1342, 1219, 1345, 1301, 1273, 1210, 1252, 120..."


In [112]:
%%time 
justBasics = just2018.reset_index()[['Season','DayNum','WTeamID','LTeamID','WScore','LScore','WTeamsPlayed','LTeamsPlayed']].copy()
for i in range(len(justBasics[4000:4010])):
    day = justBasics.loc[i+4000,'DayNum']
    tempDF = just2018.loc[just2018['DayNum'] < day].copy()
    try:
        tempDF['playedWTeam'] = tempDF.apply(lambda row: (row['WTeamID'] in row['WTeamsPlayed']) or (row['LTeamID'] in row['WTeamsPlayed']),axis=1)
        tempDF['playedLTeam'] = tempDF.apply(lambda row: (row['WTeamID'] in row['LTeamsPlayed']) or (row['LTeamID'] in row['LTeamsPlayed']),axis=1)
        wTeam = justBasics.loc[i,'WTeamID']
        lTeam = justBasics.loc[i,'LTeamID']
        tempDF['wTeamInRow'] = tempDF.apply(lambda row: (row['WTeamID'] == wTeam) or (row['LTeamID'] == wTeam),axis=1)
        tempDF['lTeamInRow'] = tempDF.apply(lambda row: (row['WTeamID'] == lTeam) or (row['LTeamID'] == lTeam),axis=1)
        lOpps = getAllAvgs(tempDF.loc[tempDF['playedLTeam']])
        lOppsMean = lOpps.mean()
        wOpps = getAllAvgs(tempDF.loc[tempDF['playedWTeam']])
        wOppsMean = wOpps.mean()
        wAvgs = getAllAvgs(tempDF.loc[tempDF['wTeamInRow']])
        wAvgs
        lAvgs = getAllAvgs(tempDF.loc[tempDF['lTeamInRow']])
    except Exception as err:
        zeros = np.zeros(14)

Wall time: 9.97 s


In [116]:
just2018.mean()

Season     2018.000000
DayNum       70.756337
WTeamID    1290.320814
WScore       79.080296
WFGM         27.579648
WFGA         57.450879
WFGM3         8.339685
WFGA3        21.439593
WFTM         15.581314
WFTA         21.397965
LTeamID    1278.125994
LScore       66.919149
LFGM         23.894727
LFGA         58.488622
LFGM3         6.917484
LFGA3        21.993895
LFTM         12.212211
LFTA         17.532655
dtype: float64

In [113]:
10*200 / 60

33.333333333333336